# CSV Files
* Measure performance of reading CSV
* Explore Alternatives to Improve Performance

# Setup

In [9]:
#conda install pandas numpy matplotlib seaborn pyarrow --yes

In [28]:
#pip install smart_open

In [27]:
import csv
import random
import timeit

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow

## Generate a CSV
* This short routine will create a file around 3 MB in size

In [17]:
rows=100000
print(f'Generating {rows} rows')

with open('sample.csv', 'w') as file:   
    columns = ['x1', 'x2', 'x3', 'x4', 'x5']
    writer  = csv.DictWriter(file, fieldnames=columns)
    writer.writerow(dict(zip(columns, columns)))
    
    x2 = ['Alpha', 'Beta', 'Gamma', 'Phi', 'Rho']
    x3 = ['John', 'Jane', 'William', 'Wilma']
    x4 = ['Atlanta', 'New York', 'London', 'Dehli']

    for index in range(rows):
        writer.writerow(dict([
            ('x1', index),
            ('x2', random.choice(x2)),
            ('x3', random.choice(x3)),
            ('x4', random.choice(x4)),
            ('x5', str(random.randint(100, 500)))
        ]))

Generating 100000 rows


In [18]:
!ls -altr

total 5904
-rw-r--r--   1 stevew  staff    16835 May  3 10:44 ProducerConsumer.ipynb
-rw-r--r--   1 stevew  staff    21297 May  3 10:44 Threads.ipynb
-rw-r--r--   1 stevew  staff    29898 May  3 10:44 Multiprocessing.ipynb
drwxr-xr-x  12 stevew  staff      408 May  3 10:46 .git
drwx------+ 24 stevew  staff      816 May  5 03:53 ..
drwxr-xr-x   6 stevew  staff      204 May  5 12:57 .ipynb_checkpoints
-rw-r--r--   1 stevew  staff     2857 May  5 12:58 CSV.ipynb
drwxr-xr-x   9 stevew  staff      306 May  5 12:58 .
-rw-r--r--   1 stevew  staff  2938882 May  5 12:59 sample.csv


# Baseline

In [24]:
%%timeit
with open('sample.csv', 'r') as file:   
    r = file.read()

10.1 ms ± 72 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Pandas

In [38]:
%%timeit
pd.read_csv('sample.csv')

74.5 ms ± 1.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Compressed
* Compression will save on the size of the file
* But, you'll pay a penalty reading the data

In [35]:
df = pd.read_csv('sample.csv')
df.to_csv('sample.csv.gz', compression='gzip')

In [37]:
!ls -al

total 9120
drwxr-xr-x  12 stevew  staff      408 May  5 13:29 .
drwx------+ 24 stevew  staff      816 May  5 03:53 ..
drwxr-xr-x  12 stevew  staff      408 May  3 10:46 .git
drwxr-xr-x   7 stevew  staff      238 May  5 13:12 .ipynb_checkpoints
-rw-r--r--   1 stevew  staff     7813 May  5 13:29 CSV.ipynb
-rw-r--r--   1 stevew  staff    29898 May  3 10:44 Multiprocessing.ipynb
-rw-r--r--   1 stevew  staff    16835 May  3 10:44 ProducerConsumer.ipynb
-rw-r--r--   1 stevew  staff      821 May  5 13:13 SmartOpen.ipynb
-rw-r--r--   1 stevew  staff    21297 May  3 10:44 Threads.ipynb
-rw-r--r--   1 stevew  staff  2938882 May  5 12:59 sample.csv
-rw-r--r--   1 stevew  staff   817656 May  5 13:28 sample.csv.gz
-rw-r--r--   1 stevew  staff   818953 May  5 13:16 sample.parquet


In [39]:
%%timeit
pd.read_csv('sample.csv.gz')

104 ms ± 824 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Parquet
* The engine will attempt to find and use pyarrow (falling back tp fastparquet)
* Changing the CSV to parquet will result in a much smaller and faster file access


In [31]:
df = pd.read_csv('sample.csv')
df.to_parquet('sample.parquet', index=False)

In [32]:
ls -altr

total 7520
-rw-r--r--   1 stevew  staff    16835 May  3 10:44 ProducerConsumer.ipynb
-rw-r--r--   1 stevew  staff    21297 May  3 10:44 Threads.ipynb
-rw-r--r--   1 stevew  staff    29898 May  3 10:44 Multiprocessing.ipynb
drwxr-xr-x  12 stevew  staff      408 May  3 10:46 .git/
drwx------+ 24 stevew  staff      816 May  5 03:53 ../
-rw-r--r--   1 stevew  staff  2938882 May  5 12:59 sample.csv
drwxr-xr-x   7 stevew  staff      238 May  5 13:12 .ipynb_checkpoints/
-rw-r--r--   1 stevew  staff      821 May  5 13:13 SmartOpen.ipynb
-rw-r--r--   1 stevew  staff     5490 May  5 13:15 CSV.ipynb
-rw-r--r--   1 stevew  staff   818953 May  5 13:16 sample.parquet
drwxr-xr-x  11 stevew  staff      374 May  5 13:16 ./


In [34]:
%%timeit
pd.read_parquet('sample.parquet')

28.7 ms ± 844 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Chunking Pandas
* Read in 10000 lines at a time
* While there is more overhead in reading from a CPU perspective, we can efficiently iterate over a large file in a memory efficient way

In [42]:
%%timeit
chunks = pd.read_csv('sample.csv', chunksize=10000)
df = pd.concat(chunks)

114 ms ± 1.56 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Random Access
* If we wanted to pull one column from a csv performing an operation on it

## Pandas CSV
* All columns must be read in from a csv file

In [56]:
df = pd.read_csv('sample.csv')
print(df.memory_usage())
df.columns

Index       128
x1       800000
x2       800000
x3       800000
x4       800000
x5       800000
dtype: int64


Index(['x1', 'x2', 'x3', 'x4', 'x5'], dtype='object')

In [53]:
%%timeit
df = pd.read_csv('sample.csv')
df.x4 * 2

94.9 ms ± 1.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Pandas Parquet
* With parquet, we can read in one column (push-down predicate)
* Offers an advantage for some use cases

In [57]:
df = pd.read_parquet('sample.parquet', columns=['x4'])
print(df.memory_usage())
df.columns

Index       128
x4       800000
dtype: int64


Index(['x4'], dtype='object')

In [47]:
%%timeit
df = pd.read_parquet('sample.parquet', columns=['x4'])
df.x4 * 2

27.6 ms ± 597 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
